In [1]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import numpy as np

In [2]:
# Load Pune boundary
pune_boundary = gpd.read_file(r"C:\Users\admin\Downloads\pune-admin-wards.geojson")

In [3]:
# Define grid spacing and create a grid of points within the boundary
grid_spacing = 0.01  # Adjust this for the density of points
minx, miny, maxx, maxy = pune_boundary.total_bounds
x_coords = np.arange(minx, maxx, grid_spacing)
y_coords = np.arange(miny, maxy, grid_spacing)
grid_points = [Point(x, y) for x in x_coords for y in y_coords]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs=pune_boundary.crs)
grid_gdf = gpd.clip(grid_gdf, pune_boundary)

# Update the tag for malls instead of cinemas
tags = {
    "amenity": ["fuel", "parking", "supermarket"],
    "building": ["commercial", "residential"],
    "landuse": ["residential", "industrial", "commercial"],
    "public_transport": "station"
}

In [4]:
# Extract features from OSM within Pune boundary
all_features = ox.geometries_from_polygon(pune_boundary.geometry[0], tags=tags)

# Separate each category of interest for individual distance calculations
fuel_stations = all_features[all_features["amenity"] == "fuel"]
parking_areas = all_features[all_features["amenity"] == "parking"]
commercial_buildings = all_features[all_features["building"] == "commercial"]
residential_buildings = all_features[all_features["building"] == "residential"]
residential_landuse = all_features[all_features["landuse"] == "residential"]
industrial_landuse = all_features[all_features["landuse"] == "industrial"]
commercial_landuse = all_features[all_features["landuse"] == "commercial"]

C:\Users\admin\AppData\Local\Temp\ipykernel_15240\3511131318.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  all_features = ox.geometries_from_polygon(pune_boundary.geometry[0], tags=tags)


In [5]:
# Print the count of features in each category
print("Number of fuel stations:", fuel_stations.shape[0])
print("Number of parking areas:", parking_areas.shape[0])
print("Number of commercial buildings:", commercial_buildings.shape[0])
print("Number of residential buildings:", residential_buildings.shape[0])
print("Number of residential landuse areas:", residential_landuse.shape[0])
print("Number of industrial landuse areas:", industrial_landuse.shape[0])
print("Number of commercial landuse areas:", commercial_landuse.shape[0])

Number of fuel stations: 5
Number of parking areas: 49
Number of commercial buildings: 92
Number of residential buildings: 120
Number of residential landuse areas: 162
Number of industrial landuse areas: 6
Number of commercial landuse areas: 9


In [6]:
# Helper function to calculate minimum distance from each grid point to a feature
def calculate_min_distance(grid, features):
    if features.empty:
        return np.nan
    return grid.geometry.apply(lambda point: features.distance(point).min())

# Calculate distances for each feature category using the updated feature list
grid_gdf["distance_to_fuel_station"] = calculate_min_distance(grid_gdf, fuel_stations)
grid_gdf["distance_to_parking"] = calculate_min_distance(grid_gdf, parking_areas)
grid_gdf["distance_to_commercial_building"] = calculate_min_distance(grid_gdf, commercial_buildings)
grid_gdf["distance_to_residential_building"] = calculate_min_distance(grid_gdf, residential_buildings)
grid_gdf["distance_to_residential_landuse"] = calculate_min_distance(grid_gdf, residential_landuse)
grid_gdf["distance_to_industrial_landuse"] = calculate_min_distance(grid_gdf, industrial_landuse)
grid_gdf["distance_to_commercial_landuse"] = calculate_min_distance(grid_gdf, commercial_landuse)

# Check the first few rows to verify the calculated distances
grid_gdf.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_15240\1168214773.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppData\Local\Temp\ipykernel_15240\1168214773.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppData\Local\Temp\ipykernel_15240\1168214773.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppD

,geometry,distance_to_fuel_station,distance_to_parking,distance_to_commercial_building,distance_to_residential_building,distance_to_residential_landuse,distance_to_industrial_landuse,distance_to_commercial_landuse
182,POINT (73.84156 18.44883),0.094687,0.091997,0.106410,0.098904,0.085823,0.097652,0.096227
221,POINT (73.86156 18.43883),0.109599,0.107620,0.125390,0.114040,0.102869,0.118152,0.113751
222,POINT (73.86156 18.44883),0.100247,0.098433,0.117267,0.104721,0.094096,0.110863,0.105078
242,POINT (73.87156 18.44883),0.104361,0.102957,0.122283,0.108897,0.099491,0.117902,0.110589
202,POINT (73.85156 18.44883),0.096992,0.094743,0.111523,0.101361,0.089498,0.104039,0.100265


In [9]:
import rasterio
import numpy as np
import geopandas as gpd

# Load Pune boundary from GeoJSON file
pune_boundary = gpd.read_file(r"C:\Users\admin\Downloads\pune-admin-wards_2017.geojson")

# Reproject the Pune boundary to the CRS of the population density raster (e.g., UTM Zone 43N)
pune_boundary = pune_boundary.to_crs(epsg=32643)  # UTM Zone 43N for Pune

# Ensure the grid and population density raster use the same CRS
grid_crs = grid_gdf.crs
population_density_raster = rasterio.open(r"C:\Users\admin\Downloads\ind_pd_2020_1km.tif")

# Check if CRS of grid and raster match; if not, reproject the grid
if grid_crs != population_density_raster.crs:
    grid_gdf = grid_gdf.to_crs(population_density_raster.crs)

# Extract population density values for each grid point
pop_density_values = []

# Ensure the grid points are within the bounds of the raster
xmin, ymin, xmax, ymax = population_density_raster.bounds
for point in grid_gdf.geometry:
    if xmin <= point.x <= xmax and ymin <= point.y <= ymax:
        # Convert the coordinates to row, col indices for the raster
        row, col = population_density_raster.index(point.x, point.y)
        
        # Safely append population density value, checking for out-of-bounds errors
        if 0 <= row < population_density_raster.height and 0 <= col < population_density_raster.width:
            # Read the population density value at the row, col
            pop_density_value = population_density_raster.read(1)[row, col]
            pop_density_values.append(pop_density_value)
        else:
            pop_density_values.append(np.nan)  # Handle out-of-bounds points
    else:
        pop_density_values.append(np.nan)  # Handle points outside the raster bounds

# Add population density to grid GeoDataFrame
grid_gdf["population_density"] = pop_density_values

# Inspect the updated grid
grid_gdf

,geometry,distance_to_fuel_station,distance_to_parking,distance_to_commercial_building,distance_to_residential_building,distance_to_residential_landuse,distance_to_industrial_landuse,distance_to_commercial_landuse,population_density
182,POINT (73.84156 18.44883),0.094687,0.091997,0.106410,0.098904,0.085823,0.097652,0.096227,6763.399902
221,POINT (73.86156 18.43883),0.109599,0.107620,0.125390,0.114040,0.102869,0.118152,0.113751,4165.090332
222,POINT (73.86156 18.44883),0.100247,0.098433,0.117267,0.104721,0.094096,0.110863,0.105078,7511.783691
242,POINT (73.87156 18.44883),0.104361,0.102957,0.122283,0.108897,0.099491,0.117902,0.110589,8083.099121
202,POINT (73.85156 18.44883),0.096992,0.094743,0.111523,0.101361,0.089498,0.104039,0.100265,8517.727539
...,...,...,...,...,...,...,...,...,...
356,POINT (73.92156 18.58883),0.088168,0.102823,0.107717,0.091917,0.094627,0.080113,0.116584,6371.844238
316,POINT (73.90156 18.58883),0.068589,0.084788,0.088213,0.073012,0.075934,0.060666,0.097085,6609.640137
376,POINT (73.93156 18.58883),0.098021,0.112077,0.117531,0.101527,0.104155,0.089928,0.126392,3271.658203
317,POINT (73.90156 18.59883),0.071565,0.089249,0.091175,0.077234,0.080427,0.064075,0.099993,8268.689453


In [13]:
import pandas as pd

# Define thresholds for high demand
fuel_station_threshold = 0.1  # Example threshold for distance to fuel stations (in km)
population_density_threshold = 5000  # Example threshold for population density (people per km²)

# Assign labels based on the thresholds
def assign_label(row):
    if row['distance_to_fuel_station'] < fuel_station_threshold and row['population_density'] > population_density_threshold:
        return 1  # High demand
    else:
        return 0  # Low demand

# Apply label function to your dataframe
grid_gdf['label'] = grid_gdf.apply(assign_label, axis=1)

# Inspect the labels
grid_gdf[['geometry', 'label']].head()

,geometry,label
182,POINT (73.84156 18.44883),1
221,POINT (73.86156 18.43883),0
222,POINT (73.86156 18.44883),0
242,POINT (73.87156 18.44883),0
202,POINT (73.85156 18.44883),1


In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Handling missing values (if any)
grid_gdf.fillna(0, inplace=True)

# Selecting features for model input
feature_columns = ['distance_to_fuel_station', 'distance_to_parking',
                   'distance_to_commercial_building', 'distance_to_residential_building',
                   'distance_to_residential_landuse', 'distance_to_industrial_landuse',
                   'distance_to_commercial_landuse', 'population_density']

X = grid_gdf[feature_columns]
y = grid_gdf['label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9767441860465116
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.97      1.00      0.98        29

    accuracy                           0.98        43
   macro avg       0.98      0.96      0.97        43
weighted avg       0.98      0.98      0.98        43



In [21]:
import folium
from folium.plugins import MarkerCluster

# Predict for all grid points
grid_gdf['predicted_label'] = model.predict(X_scaled)

grid_gdf['predicted_label'].head()

182    1
221    0
222    0
242    0
202    1
Name: predicted_label, dtype: int64

In [39]:
import numpy as np
from geopy.distance import geodesic
from shapely.geometry import Point
from scipy.spatial import cKDTree

# Step 1: Filter grid points with high population density and positive prediction
high_demand_points = grid_gdf[(grid_gdf['predicted_label'] == 1) & (grid_gdf['population_density'] > 6000)]

# Extract latitude and longitude of filtered points
high_demand_points['coords'] = high_demand_points['geometry'].apply(lambda geom: (geom.y, geom.x))
high_demand_coords = high_demand_points['coords'].tolist()

# Step 2: Enforce a minimum distance of 3 km between points using a KD-Tree
stations_to_plot = []
if high_demand_coords:
    tree = cKDTree(high_demand_coords)
    selected_indices = []

    for i, point in enumerate(high_demand_coords):
        # Check if the point is at least 3 km away from previously selected points
        if all(geodesic(point, high_demand_coords[j]).km >= 2.5 for j in selected_indices):
            selected_indices.append(i)
    
    # Get final selected locations
    stations_to_plot = [high_demand_coords[i] for i in selected_indices]

# Display the selected EV station locations
print("Predicted EV Station Locations (Latitude, Longitude):")
for location in stations_to_plot:
    print(location)

C:\Users\admin\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Predicted EV Station Locations (Latitude, Longitude):
(18.448828616000004, 73.84156098600005)
(18.468828616000007, 73.82156098600004)
(18.48882861600001, 73.84156098600005)
(18.458828616000005, 73.87156098600006)
(18.48882861600001, 73.87156098600006)
(18.47882861600001, 73.90156098600008)
(18.49882861600001, 73.81156098600003)
(18.518828616000015, 73.84156098600005)
(18.518828616000015, 73.87156098600006)
(18.528828616000016, 73.90156098600008)
(18.54882861600002, 73.88156098600007)
(18.538828616000018, 73.79156098600002)
(18.54882861600002, 73.82156098600004)
(18.538828616000018, 73.9315609860001)
(18.55882861600002, 73.91156098600008)
(18.568828616000022, 73.78156098600002)
(18.54882861600002, 73.85156098600005)
(18.578828616000024, 73.88156098600007)
(18.588828616000026, 73.92156098600009)


In [40]:
import folium
from folium.plugins import MarkerCluster

# Initialize the map centered around Pune
m = folium.Map(location=[18.5204, 73.8567], zoom_start=12)

# Create a marker cluster for visualization
marker_cluster = MarkerCluster().add_to(m)

# Plot the selected EV station locations on the map
for idx, (lat, lon) in enumerate(stations_to_plot):
    folium.Marker(
        location=[lat, lon],
        popup=f"Predicted EV Station {idx+1}",
        icon=folium.Icon(color="green", icon="bolt", prefix="fa")
    ).add_to(marker_cluster)

# Display the map (in Jupyter, or save to an HTML file)
m

In [41]:
import pandas as pd

# Convert selected locations to a DataFrame
stations_df = pd.DataFrame(stations_to_plot, columns=['Latitude', 'Longitude'])

# Save DataFrame to a CSV file
stations_df.to_csv('predicted_ev_station_locations.csv', index=False)

print("Locations saved to 'predicted_ev_station_locations.csv'")

Locations saved to 'predicted_ev_station_locations.csv'
